# OPAN5505 Lab 3 Assignment

## Background

You've been engaged by the Federal Aviation Administration to perform an analysis on "strike" data. A "strike" refers to when an aircraft and an object within the air (typically wildlife) collide.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Prepare your environment

You will need the `polars` package for this assignment. We'll also use `datetime` for date handling.

In [1]:
# Import required libraries
import polars as pl
import datetime

## Load your dataset

Read the `faa_strikes.txt` dataset into Polars. Name the resulting DataFrame `strikes`.

In [7]:
# Read the faa_strikes.txt data
strikes = pl.read_csv('/content/faa_strikes.txt', truncate_ragged_lines=True, separator=' ')
display(strikes.head())
print(strikes.columns)

Airport:,Code Airport:,Name Origin,State Origin,State,Code Country Aircraft:,Type Aircraft:,Number,of,engines Collision,Date,and,Time When:,Time,of_duplicated_0,day When:,Phase,of_duplicated_1,flight Effect:,Amount,of_duplicated_2,damage,(detailed) Effect:,Impact,to,flight Effect:_duplicated_0,Indicated,Damage Cost:,Aircraft,time,out,of_duplicated_3,service,(hours) Cost:,Total,$ Days Feet,above,ground Miles,from,airport Wildlife:,Animal,Category Wildlife:,Species,Order Wildlife:,Species_duplicated_0,Group Wildlife:,Species Wildlife:,Species_duplicated_1,ID Number,of_duplicated_4,Strikes Record,ID
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""KAAF APALACHICOLA""","""REGIONAL""","""ARPT Florida FL United""","""States NA NA 2012-09-20T23:30:…","""run None None No""","""damage NA 0 NA 0 NA Terrestria…","""Mammals Canids Wolves,""","""Dogs,""","""Foxes,""","""Coyote Domestic""","""dog 1F12 1 17459""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""KAAF APALACHICOLA""","""REGIONAL""","""ARPT Florida FL United""","""States Airplane 1 2013-04-23T2…","""run None None No""","""damage NA 0 NA NA NA Birds Pel…","""Herons,""","""Egrets,""","""Bitterns,""","""Ibises Herons,""","""Egrets,""","""Bitterns Herons,""","""egrets,""","""bitterns I1 1 17114""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""KABE LEHIGH""","""VALLEY""","""INTL Pennsylvania PA United""","""States Airplane 2 2009-04-23T1…","""run Medium Aborted""","""Take-off Caused""","""damage 720 171132 30 0 0 Birds…","""Falcons Caracaras,""","""Falcons American""","""kestrel K5114 1 259361""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""KABE LEHIGH""","""VALLEY""","""INTL Pennsylvania PA United""","""States Airplane 2 2014-04-14T0…","""damage 15 600 1 NA NA Birds Pi…","""and""","""Doves Doves Mourning""","""dove O2205 1 345167""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""KABE LEHIGH""","""VALLEY""","""INTL Pennsylvania PA United""","""States Airplane 2 2009-03-31T2…","""damage 12 188245 1 NA 0 Birds …","""Kites,""","""Eagles,""","""Ospreys,""","""Vultures Kites,""","""Hawks,""","""Eagles Red-tailed""","""hawk K3302 1 262782""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


['Airport:', 'Code\tAirport:', 'Name\tOrigin', 'State\tOrigin', 'State', 'Code\tCountry\tAircraft:', 'Type\tAircraft:', 'Number', 'of', 'engines\tCollision', 'Date', 'and', 'Time\tWhen:', 'Time', 'of_duplicated_0', 'day\tWhen:', 'Phase', 'of_duplicated_1', 'flight\tEffect:', 'Amount', 'of_duplicated_2', 'damage', '(detailed)\tEffect:', 'Impact', 'to', 'flight\tEffect:_duplicated_0', 'Indicated', 'Damage\tCost:', 'Aircraft', 'time', 'out', 'of_duplicated_3', 'service', '(hours)\tCost:', 'Total', '$\tDays\tFeet', 'above', 'ground\tMiles', 'from', 'airport\tWildlife:', 'Animal', 'Category\tWildlife:', 'Species', 'Order\tWildlife:', 'Species_duplicated_0', 'Group\tWildlife:', 'Species\tWildlife:', 'Species_duplicated_1', 'ID\tNumber', 'of_duplicated_4', 'Strikes\tRecord', 'ID']


## Question 1: Running total of strikes by day through 2013

You are interested in seeing the running total of strikes over time (`Collision Date and Time`) on a daily basis. Using the `strikes` dataframe, create a new column that removes the time information but keeps the date information from the `Collision Date and Time` column (call this new field `date`). Next, aggregate the `Number of Strikes` by day (call this new field `daily_strikes`). Sort the data in ascending order by `date`. Then, create the running total of `daily_strikes` (name the new field `strikes_cumulative`) and filter to only the records **up to and including** the `date` of 2013-12-31. Name the resulting data frame `running_total_strikes`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [28]:
running_total_strikes = strikes.with_columns(
    pl.col("Date").str.extract(r"(\d{4}-\d{2}-\d{2})").cast(pl.Date).alias("date")
)
display(running_total_strikes.head())

Airport:,Code Airport:,Name Origin,State Origin,State,Code Country Aircraft:,Type Aircraft:,Number,of,engines Collision,Date,and,Time When:,Time,of_duplicated_0,day When:,Phase,of_duplicated_1,flight Effect:,Amount,of_duplicated_2,damage,(detailed) Effect:,Impact,to,flight Effect:_duplicated_0,Indicated,Damage Cost:,Aircraft,time,out,of_duplicated_3,service,(hours) Cost:,Total,$ Days Feet,above,ground Miles,from,airport Wildlife:,Animal,Category Wildlife:,Species,Order Wildlife:,Species_duplicated_0,Group Wildlife:,Species Wildlife:,Species_duplicated_1,ID Number,of_duplicated_4,Strikes Record,ID,date
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date
"""KAAF APALACHICOLA""","""REGIONAL""","""ARPT Florida FL United""","""States NA NA 2012-09-20T23:30:…","""run None None No""","""damage NA 0 NA 0 NA Terrestria…","""Mammals Canids Wolves,""","""Dogs,""","""Foxes,""","""Coyote Domestic""","""dog 1F12 1 17459""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""KAAF APALACHICOLA""","""REGIONAL""","""ARPT Florida FL United""","""States Airplane 1 2013-04-23T2…","""run None None No""","""damage NA 0 NA NA NA Birds Pel…","""Herons,""","""Egrets,""","""Bitterns,""","""Ibises Herons,""","""Egrets,""","""Bitterns Herons,""","""egrets,""","""bitterns I1 1 17114""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""KABE LEHIGH""","""VALLEY""","""INTL Pennsylvania PA United""","""States Airplane 2 2009-04-23T1…","""run Medium Aborted""","""Take-off Caused""","""damage 720 171132 30 0 0 Birds…","""Falcons Caracaras,""","""Falcons American""","""kestrel K5114 1 259361""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""KABE LEHIGH""","""VALLEY""","""INTL Pennsylvania PA United""","""States Airplane 2 2014-04-14T0…","""damage 15 600 1 NA NA Birds Pi…","""and""","""Doves Doves Mourning""","""dove O2205 1 345167""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""KABE LEHIGH""","""VALLEY""","""INTL Pennsylvania PA United""","""States Airplane 2 2009-03-31T2…","""damage 12 188245 1 NA 0 Birds …","""Kites,""","""Eagles,""","""Ospreys,""","""Vultures Kites,""","""Hawks,""","""Eagles Red-tailed""","""hawk K3302 1 262782""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


## Question 2: States with the third highest financial cost

The FAA is interested in the financial cost of strikes for each `Origin State`. Using the `strikes` dataframe: first, sum `Cost: Total $` by `Origin State` (name the resulting column `damage`) then `rank` these states in order of which state had the most financial `damage` (name the new column with rank information, `ranking`). Next, filter to show the row with the third highest `ranking`. Use a window function to answer this question. Name the resulting dataframe `damage_state`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [32]:
damage_state = strikes.group_by("State").agg(
    pl.col("Total").cast(pl.Float64).sum().alias("damage")
).with_columns(
    pl.col("damage").rank(method="min").alias("ranking")
).filter(pl.col("ranking") == 3)
display(damage_state)

State,damage,ranking
str,f64,u32


## Question 3: What are the second costliest Species Groups for each Aircraft Type?

The FAA wants to know if some species groups are more dangerous to particular types of aircraft. Using the `strikes` dataframe: first, sum financial damage (`Cost: Total $`) information by `Aircraft: Type` and `Wildlife: Species Group` (name the new field `damage`). Rank the rows within each `Aircraft: Type` based on the greatest amount of `damage`; when performing the ranking function, name the new column `ranking`. Return the rows that represent the `Wildlife: Species Group` values that caused the **second most** financial `damage` to each `Aircraft: Type`. Name the resulting dataframe `type_species`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a ranking function to receive credit on this question.*

In [34]:
type_species = strikes.group_by(["Type\tAircraft:", "Species_duplicated_0 Group"]).agg(
    pl.col("Total $\tDays\tFeet").cast(pl.Float64, strict=False).sum().alias("damage")
).with_columns(
    pl.col("damage").rank(method="dense").over("Type\tAircraft:", order_by=pl.col("damage"), descending=True).alias("ranking")
).filter(pl.col("ranking") == 2)
display(type_species)

ColumnNotFoundError: Species_duplicated_0 Group

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["Airport:", "Code	Airport:", "Name	Origin", "State	Origin", ...]; PROJECT */52 COLUMNS

## Question 4: Which days had the greatest positive jump in strikes?

The FAA wants to investigate which days had the largest increase in strikes from the previous day. Using the `strikes` dataframe: sum the `Number of Strikes` measure by day (note: not by day and time). You can use a similar technique from Question 1 to create a new column from `Collision Date and Time` without time information and with date information (name this column `date`). Next, compute the previous day's strikes using a window function (name the new column `previous_day`), and calculate the difference of strikes between the current day and the previous day (name the new column `delta_strikes`). Sort the resulting data by `delta_strikes` in descending order to find which days had the highest increase of strikes from the previous day. Name the resulting dataframe `greatest_strike_increase`.

**Hint:** This code is more involved that other exercises, so I will explicitly name the steps to answer the question.

1. Start with the `strikes` data frame
2. Create a new column called `date` which removes the time information in the `Collision Date and Time` column
3. Sum the `Number of Strikes` by `date`, and name the new column `daily_strikes`
4. Sort by the `date` column in ascending order
5. Create a new column called `previous_day` to calculate what the `daily_strikes` were in the previous day
6. Create a new column called `delta_strikes` which subtracts `previous_day` from `daily_strikes`
7. Sort in descending order by `delta_strikes`

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [31]:
# 1. Start with the strikes data frame (already loaded as 'strikes')

# 2. Create a new column called date which removes the time information in the Collision Date and Time column
strikes_with_date = strikes.with_columns(
    pl.col("Date").str.extract(r"(\d{4}-\d{2}-\d{2})").cast(pl.Date).alias("date")
).filter(pl.col("date").is_not_null()) # Filter out rows with null dates


# 3. Sum the Number of Strikes by date, and name the new column daily_strikes
daily_strikes_df = strikes_with_date.group_by("date").agg(
    pl.col("Number").cast(pl.Int64, strict=False).sum().alias("daily_strikes")
)

# Display the daily strikes before sorting
display(daily_strikes_df.head())


# 4. Sort by the date column in ascending order
sorted_daily_strikes = daily_strikes_df.sort("date")

# Display the daily strikes after sorting
display(sorted_daily_strikes.head())


# 5. Create a new column called previous_day to calculate what the daily_strikes were in the previous day
# 6. Create a new column called delta_strikes which subtracts previous_day from daily_strikes
greatest_strike_increase = sorted_daily_strikes.with_columns(
    pl.col("daily_strikes").shift(1).alias("previous_day")
).with_columns(
    (pl.col("daily_strikes") - pl.col("previous_day")).alias("delta_strikes")
)

# 7. Sort in descending order by delta_strikes
greatest_strike_increase = greatest_strike_increase.sort("delta_strikes", descending=True)

# Display the resulting dataframe
display(greatest_strike_increase)

date,daily_strikes
date,i64


date,daily_strikes
date,i64


date,daily_strikes,previous_day,delta_strikes
date,i64,i64,i64


## Question 5: Which single day had the greatest increase in strikes for each `Aircraft: Type`?

FAA was interested in the exercise from the last question but now wants to determine the largest delta between days for each `Aircraft: Type`. What are the days that had largest positive change in strikes for each `Aircraft: Type`? In order to answer this question, you will need to perform the exercise from Question 4, but this time include a grouping by `Aircraft: Type`. After grouping by `Aircraft: Type`, add a column called `ranking` and use a window function to determine the day with the largest increase in strikes from the previous day. Name the resulting data frame `greatest_strike_increase_type`

NOTE: Helicopters are not struck with a high frequency and will not show up in your analysis; this is fine.

In [29]:
# 1. Start with the strikes data frame (already loaded as 'strikes')

# 2. Create a new column called date which removes the time information in the Collision Date and Time column
strikes_with_date_type = strikes.with_columns(
    pl.col("Date").str.extract(r"(\d{4}-\d{2}-\d{2})").cast(pl.Date).alias("date")
)

# 3. Sum the Number of Strikes by date and Aircraft: Type, and name the new column daily_strikes
daily_strikes_type_df = strikes_with_date_type.group_by(["date", "Type\tAircraft:"]).agg(
    pl.col("Number").cast(pl.Int64, strict=False).sum().alias("daily_strikes")
)

# 4. Sort by Aircraft: Type and date column in ascending order
sorted_daily_strikes_type = daily_strikes_type_df.sort(["Type\tAircraft:", "date"])

# 5. Create a new column called previous_day to calculate what the daily_strikes were in the previous day,
#    partitioned by Aircraft: Type
# 6. Create a new column called delta_strikes which subtracts previous_day from daily_strikes
delta_strikes_type = sorted_daily_strikes_type.with_columns(
    pl.col("daily_strikes").shift(1).over("Type\tAircraft:").alias("previous_day")
).with_columns(
    (pl.col("daily_strikes") - pl.col("previous_day")).alias("delta_strikes")
)

# 7. Add a column called ranking to determine the day with the largest increase in strikes from the previous day,
#    partitioned by Aircraft: Type and sorted by delta_strikes in descending order.
greatest_strike_increase_type = delta_strikes_type.with_columns(
    pl.col("delta_strikes").rank(method="dense").over(["Type\tAircraft:"], order_by=pl.col("delta_strikes"), descending=True).alias("ranking")
).filter(pl.col("ranking") == 1) # Filter for the day with the largest increase (rank 1)

# Display the resulting dataframe
display(greatest_strike_increase_type)

date,Type Aircraft:,daily_strikes,previous_day,delta_strikes,ranking
date,str,i64,i64,i64,u32
